In [11]:
import jax
from jax import lax, random, numpy as jnp
from jax.tree_util import tree_flatten, tree_unflatten

import flax
from flax import linen as nn

import sympy as sy
from sympy.core.rules import Transform
import numpy as np

import sys
sys.path.append("..")
sys.path.append("../../orient/")


from eql.eqlearner import EQL, EQLdiv
from eql.symbolic import get_symbolic_expr_div, get_symbolic_expr
from eql.np_utils import flatten, unflatten


import optax
import scipy
from functools import partial
import matplotlib.pyplot as plt


In [12]:
funs = ['mul', 'cos' , 'sin', 'id', 'id', 'id', 'id']*3
e = EQLdiv(n_layers=2, functions=funs, features=1)
key = random.PRNGKey(0)

In [13]:
N = 10000
xdim = 4
x = (random.uniform(key, (N, xdim))-.5) * 2
#x = np.array([[1., 2.]]).T
#x = np.linspace(-1, 1, N)[:,None]
#y = x[:,0] + jnp.cos(x[:,1])
#y = (jnp.cos(x) + 1 - x**2)/(x-3)**3


#y = np.sin(np.pi * x[:,0])/(x[:,1]**2 + 1)
y = 1./3. * ((1.+x[:,1])*np.sin(np.pi*x[:,0]) + x[:,1]*x[:,2]*x[:,3])
#plt.scatter(x[:,0], x[:,1], c=y)

In [14]:
params = e.init({'params':key}, x, 1.0);

In [15]:
def mse_fn(params, threshold):
    pred, _ = e.apply(params, x, threshold)
    return jnp.mean((pred-y)**2)

def mse_b_fn(params, threshold):
    pred, b = e.apply(params, x, threshold)
    return jnp.mean((pred-y)**2), b

def mse_b_y_fn(params, threshold):
    pred, b = e.apply(params, x, threshold)
    return jnp.mean((pred-y)**2), b, pred


def get_mask_spec(thresh, params):
    flat, spec = tree_flatten(params)
    mask = [jnp.abs(f) > thresh for f in flat]
    return mask, spec

def apply_mask(mask, spec, params):
    flat, _ = tree_flatten(params)
    masked_params = tree_unflatten(spec, [f*m for f,m in zip(flat, mask)])
    return masked_params


def get_masked_mse(thresh, params):
    mask, spec = get_mask_spec(thresh, params)
    def masked_mse(params, threshold):
        masked_params = apply_mask(mask, spec, params)
        return mse_fn(masked_params, threshold)
    return jax.jit(masked_mse)
    

def l1_fn(params):
    return sum(
        jnp.abs(w).mean() for w in jax.tree.leaves(params["params"])
    )

def reg_fn(threshold, b):
    return (jnp.maximum(0, threshold - b)).sum()

def penalty_fn(y, B=10, supp=3):
    penalty_fn.key, _ = random.split(key)
    xr = (random.uniform(penalty_fn.key, (N, xdim))-.5) * supp
    return jnp.sum(jnp.maximum(y-B, 0)+jnp.maximum(-y-B, 0))
penalty_fn.key = key

In [16]:
def get_loss(lamba):
    def loss_fn(params, threshold):
        mse, b = mse_b_fn(params, threshold)
        return mse  + lamba * l1_fn(params) + reg_fn(threshold, b)
    return loss_fn

def get_loss_pen():
    def loss_fn(params, threshold):
        mse, b, y = mse_b_y_fn(params, threshold)
        return penalty_fn(y) + reg_fn(threshold, b)
    return loss_fn

def get_loss_grad(lamba=1e-3, is_penalty=False):
    if is_penalty:
        loss = get_loss_pen()
    else:
        loss = get_loss(lamba)
    return jax.jit(jax.value_and_grad(loss))

In [17]:
tx = optax.adam(learning_rate=1e-4)
opt_state = tx.init(params)

In [18]:
loss_grad_pen = get_loss_grad(is_penalty=True)
loss_grad_1 = get_loss_grad(0)
loss_grad_2 = get_loss_grad(1e-1)

In [19]:
def do_step(loss_grad, params, theta, opt_state):
    loss_val, grad = loss_grad(params, theta)
    updates, opt_state = tx.update(grad, opt_state)
    return optax.apply_updates(params, updates), opt_state, loss_val

In [20]:
T1 = 10_000
Tpenalty = 500
for i in range(20_000):
    theta = 1./jnp.sqrt(i/1 + 1)
    if i < T1:
        lg = loss_grad_1
    elif i >= T1:
        lg = loss_grad_2
    params, opt_state, loss_val = do_step(lg, params, theta, opt_state)
    if i % 99 == 0:
        print(loss_val, theta)
        for j in range(100):
            params, opt_state, loss_val = do_step(loss_grad_pen, params, theta, opt_state)

0.077310786 1.0
0.80495703 0.1
0.47056052 0.07088812
0.20019788 0.057928447
0.077298224 0.050188564
0.047987055 0.044901326
0.0420539 0.040996004
0.039307013 0.037959483
0.03724099 0.035511043
0.035607282 0.033482477
0.034350876 0.031766046
0.033420693 0.030289127
0.032741465 0.029000739
0.032223936 0.02786391
0.031790167 0.026851078
0.031389233 0.02594123
0.030995104 0.02511802
0.030596368 0.02436851
0.030187927 0.023682324
0.02976662 0.023051023
0.029328851 0.022467656
0.028868372 0.02192645
0.028373322 0.021422561
0.027822234 0.020951888
0.027178686 0.020510932
0.026383417 0.020096697
0.025341533 0.019706586
0.023909127 0.019338345
0.02191003 0.018990004
0.019235995 0.018659834
0.016030123 0.018346308
0.012739975 0.018048072
0.009905708 0.017763922
0.007880102 0.017492786
0.0066203796 0.017233696
0.0058044344 0.016985789
0.0051523703 0.016748281
0.0045291274 0.016520465
0.0038940085 0.016301699
0.0032633583 0.0160914
0.0026895439 0.015889037
0.002217258 0.01569412
0.0018428909 0.015

In [21]:
T = 0
for i in range(10_000):
    theta = 1./jnp.sqrt(T/1 + 1)
    loss_val, grads = loss_grad_1(params, theta)
    updates, opt_state = tx.update(grads, opt_state)
    params = optax.apply_updates(params, updates)
    T +=1 
    if i % 99 == 0:
        print(loss_val, theta)

for i in range(10_000):
    theta = 1./jnp.sqrt(T/1 + 1)
    loss_val, grads = loss_grad_2(params, theta)
    updates, opt_state = tx.update(grads, opt_state)
    params = optax.apply_updates(params, updates)
    T +=1
    if i % 99 == 0:
        print(loss_val, theta)
        print(l1_fn(params))

9880.333 1.0
0.06813525 0.1
0.05955147 0.07088812
0.053344823 0.057928447
0.048838962 0.050188564
0.045571376 0.044901326
0.043211363 0.040996004
0.041515704 0.037959483
0.04030261 0.035511043
0.039435495 0.033482477
0.03881223 0.031766046
0.038357116 0.030289127
0.03801466 0.029000739
0.037745245 0.02786391
0.037521046 0.026851078
0.037323102 0.02594123
0.037138835 0.02511802
0.036960013 0.02436851
0.036781374 0.023682324
0.036599606 0.023051023
0.03641269 0.022467656
0.036219377 0.02192645
0.03601882 0.021422561
0.035810493 0.020951888
0.035593964 0.020510932
0.03536882 0.020096697
0.035134807 0.019706586
0.034891553 0.019338345
0.034638822 0.018990004
0.03437636 0.018659834
0.034103945 0.018346308
0.03382143 0.018048072
0.033528708 0.017763922
0.03322591 0.017492786
0.03291338 0.017233696
0.03259161 0.016985789
0.03226139 0.016748281
0.031923834 0.016520465
0.031580273 0.016301699
0.031232363 0.0160914
0.03088206 0.015889037
0.030531762 0.01569412
0.030183975 0.015506205
0.029841306

In [22]:
# thr = 1e-3
# loss_grad_masked = jax.jit(jax.value_and_grad(get_masked_mse(thr, params)))
# mask, spec = get_mask_spec(thr, params)

# for i in range(1000):
#     theta = 1./jnp.sqrt(T/1 + 1)
#     loss_val, grads = loss_grad_masked(params, theta)
#     updates, opt_state = tx.update(grads, opt_state)
#     params = optax.apply_updates(params, updates)
#     T +=1
#     if i % 99 == 0:
#         print(loss_val)

In [23]:
#symb = get_symbolic_expr_div(apply_mask(mask, spec, params), funs)[0]
#symb = get_symbolic_expr_div(params, funs)[0]
#symb

In [24]:
spec, fparam = flatten(params)
full_shape = fparam.shape
mask = jnp.abs(fparam) > 0.01
idxs = jnp.arange(fparam.shape[0])[mask]
count = sum(mask).item()

In [25]:
def red_loss_grad_fn(red_param):
    full_param = jnp.zeros(full_shape).at[idxs].set(red_param)
    full_param = unflatten(spec, full_param)

    #return mse_fn(full_param, 1e-4)
    loss, grad = loss_grad_1(full_param, 1e-4)
    _, grad = flatten(grad)
    return loss, np.array(grad)[idxs,]
    
#red_loss_grad = jax.jit(jax.value_and_grad(red_mse_fn))

In [26]:
x0, f, info = scipy.optimize.fmin_l_bfgs_b(
        red_loss_grad_fn,
        x0 = np.array(fparam[mask]),
        factr=1.,
        m=500,
        pgtol=1e-13,
        maxls=100)
#x0[np.abs(x0) < 1e-3] = 0.0

In [27]:
f

5.432953548734076e-05

In [28]:
final_param = unflatten(spec, jnp.zeros(full_shape).at[idxs].set(x0))

In [29]:
symb = get_symbolic_expr_div(final_param, funs)[0]
#symb

In [30]:
def clean_expr(expr):
    # WARNING: might return 0/NaN/inf if expression only contains small numbers
    def prune(expr, thr=1e-5):
        return expr.replace(lambda x: x.is_Number and abs(x) < thr, lambda x: 0)
    
    def rounding(expr, dig=3):
        return expr.xreplace(Transform(lambda x: x.round(dig), lambda x: x.is_Number))
    
    # prune small numbers
    expr = prune(expr)
    # round number
    expr = rounding(expr)
    # expand
    expr = prune(sy.expand(expr), 1e-3)
    return sy.simplify(expr)

In [31]:
clean_expr(symb)

(0.00116*x0**2 + 0.00152*x0*x1 + 0.043*x0*sin(0.939*x0 - 0.042*x1) + 0.00455*x0*cos(0.851*x0) - 0.0083*x0*cos(0.329*x0 - 0.011*x1) + 0.00563*x0 - 0.003*x1*x2*x3 + 0.0139*x1*sin(0.939*x0 - 0.042*x1) - 0.00109*x1*cos(0.329*x0 - 0.011*x1) - 0.00141*x1 + 0.193*sin(0.939*x0 - 0.042*x1)**2 + 0.0227*sin(0.939*x0 - 0.042*x1)*cos(0.851*x0) - 0.0286*sin(0.939*x0 - 0.042*x1)*cos(0.329*x0 - 0.011*x1) - 0.00538*sin(0.939*x0 - 0.042*x1) - 0.192*sin(0.618*x0 + 0.049*x1 + 0.617*sin(0.939*x0 - 0.042*x1) - 0.543*cos(0.851*x0) - 0.323) + 0.00251*cos(0.851*x0)*cos(0.329*x0 - 0.011*x1) - 0.00544*cos(0.851*x0) - 0.00424*cos(0.329*x0 - 0.011*x1)**2 - 0.136*cos(0.329*x0 - 0.011*x1) - 0.266*cos(0.515*x0 + 0.021*x1 + 0.091*sin(0.939*x0 - 0.042*x1) + 0.754*cos(0.851*x0) + 0.61*cos(0.329*x0 - 0.011*x1) + 0.556) - 0.092)/(0.00312*x0*sin(0.939*x0 - 0.042*x1) + 0.009*x0 + 0.00101*x1*sin(0.939*x0 - 0.042*x1) + 0.014*sin(0.939*x0 - 0.042*x1)**2 + 0.00164*sin(0.939*x0 - 0.042*x1)*cos(0.851*x0) - 0.00207*sin(0.939*x0 - 

In [32]:
f

5.432953548734076e-05

In [33]:
symb


(-0.00570825677272162*x0 - 0.000128824753874212*x1**2 + 0.00264956248542309*x1*x2*x3 + 0.00124893118810871*x1 + 0.331002801656723*(0.0246044685592219*x0 + 0.0280999725037983*x1 + 0.797916173934937*sin(0.938977181911469*x0 - 0.0416975133121014*x1) + 0.0991253554821014*cos(0.850990653038025*x0) - 0.190701335668564*cos(0.329028427600861*x0 - 0.0107447095215321*x1))*(-0.140399995190876*x0 - 0.0272828772878377*x1 - 0.732174277305603*sin(0.938977181911469*x0 - 0.0416975133121014*x1) + 0.00529762217774987*cos(0.850990653038025*x0) - 0.0670695826411247*cos(0.329028427600861*x0 - 0.0107447095215321*x1) + 0.0431494787335396) - 0.00593531414703319*sin(0.938977181911469*x0 - 0.0416975133121014*x1) + 0.191647157073021*sin(0.617506785761334*x0 + 0.000285434250866177*x1**2 + 0.0487467066397896*x1 + 0.616536378860474*sin(0.938977181911469*x0 - 0.0416975133121014*x1) - 0.54260927438736*cos(0.850990653038025*x0) - 0.322575390338898) + 0.00437627614265201*cos(0.850990653038025*x0) + 0.139458716833659*cos